# ỨNG DỤNG CLUSTERING ĐỂ PHÁT HIỆN ẢNH TRÙNG

1. Yêu cầu chung: Dùng kỹ thuật clustering để tạo công cụ hỗ trợ phát hiện các ảnh trùng nhau

2. Yêu cầu cụ thể:
  - Input: Danh sách các ảnh được lưu trong tập tin, ví dụ MotocycleDataset-Splits-1-Train.csv (xem mô tả https://colab.research.google.com/drive/1gf0GzvW0tHddKtuvMUNIvglUT-J6oW6S?usp=sharing)
  - Output: Danh sách các clusters và hiển thị các ảnh trong cluster

3. Hướng dẫn:
  - Bước 1:
    - Mỗi ảnh cần thực hiện bước rút trích đặc trưng (feature extraction), biểu diễn dưới dạng một vector đặc trưng d chiều (d-dimension).
    - Có nhiều công cụ hỗ trợ bước rút trích đặc trưng, trong bài tập này, chúng ta sẽ chọn một công cụ sao cho tốc độ xử lý nhanh nhưng kết quả tốt. Các mô hình MobileNet (https://keras.io/api/applications/mobilenet/) có thể được dùng vì đáp ứng các tiêu chí này.
  - Bước 2:
    - Chọn một thuật toán clustering - ví dụ K-Means (số lượgn clusters K=5)
    - Ghi kết quả clustering ra tập tin - thay CategoryID bằng ClusterID
  - Bước 3:
    - Hiển thị kết quả clustering - kế thừa kết quả của bài tập Hiển thị dữ liệu https://colab.research.google.com/drive/1rHbKlJd7O9E49SsJlHnZNKcyTbwXT_Ls?usp=sharing
    - Từ kết quả hiển thị, nếu các ảnh nhìn trùng nhau, nhưng tên tập tin khác nhau thì có thể đưa vào danh sách hậu kiểm.

#LỚP: CS114.O21

---


#Danh sách thành viên:


1.   22521266 : Trần Giang Sử
2.   22521332 : Nguyễn Dương Quốc Thắng
3.   22521402 : Đinh Quốc Thịnh


---



#NGÀY CẬP NHẬT: 28/6/2024

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Extract features


In [2]:
import tensorflow as tf
from keras.applications import MobileNetV2
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Thông số
IMG_SIZE = 224
BATCH_SIZE = 32

# Đường dẫn đến file CSV chứa dữ liệu
df_path = '/content/drive/MyDrive/Public/MotocycleDataset-Splits-1-Test.csv'

# Đọc file CSV
df = pd.read_csv(df_path, header=None, names=['ImageFullPath', 'Category'])

# Hiển thị các cột của DataFrame để kiểm tra
print(df.columns)

# Chuẩn bị ImageDataGenerator để chuẩn hóa ảnh
datagen = ImageDataGenerator(rescale=1./255)

# Hàm load và preprocess ảnh
def load_and_preprocess_image(img_path):
    try:
        img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = datagen.standardize(img)
        return img
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None

def extract_features(df):
    features = []
    img_path_list = []
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    feature_extractor = tf.keras.Model(inputs, x)
    for index, row in df.iterrows():
        img_path = f'/content/drive/MyDrive/Public/{row["ImageFullPath"]}'
        img = load_and_preprocess_image(img_path)
        if img is not None and img.shape == (1, IMG_SIZE, IMG_SIZE, 3):
            features.append(feature_extractor(img))
    if len(features) == 0:
        raise ValueError("No images were loaded successfully. Please check the file paths and image files.")
    return features


# Trích xuất đặc trưng từ dữ liệu
try:
    features = extract_features(df)
except ValueError as e:
    print(e)
    exit()

Index(['ImageFullPath', 'Category'], dtype='object')
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1027.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1027.jpg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1098.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1098.jpg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1116.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1116.jpg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1166.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.17.jpeg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.17.jpeg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.21.jpeg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.21.jpeg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.30.jpeg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.30.jpeg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.37.jpeg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.37.jpeg'


#ViSuaLize


In [4]:
image_path_list = []
for index, row in df.iterrows():
        img_path = f'/content/drive/MyDrive/Public/{row["ImageFullPath"]}'
        img = load_and_preprocess_image(img_path)
        if img is not None and img.shape == (1, IMG_SIZE, IMG_SIZE, 3):
          image_path_list.append(img_path)

Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1027.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1027.jpg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1098.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1098.jpg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1116.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1116.jpg'
Error loading image /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1166.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.11

In [9]:
import csv
from sklearn.cluster import KMeans
from collections import defaultdict


features = np.array(features)

# Phân cụm ảnh bằng K-Means
num_clusters = 5  # Thay đổi số lượng cụm theo ý muốn
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(features)
labels = kmeans.labels_


# Lưu thông tin về các hình ảnh và nhãn cụm vào một dictionary
cluster_dict = defaultdict(list)
for i, label in enumerate(labels):
    cluster_dict[label].append(image_path_list[i])



print("Các hình ảnh có đặc trưng giống nhau:")
for cluster_id, image_paths in cluster_dict.items():
    print(f"Cụm {cluster_id}:")
    visited = set()  # Set để lưu trữ các đặc trưng đã xem qua
    for i in range(len(image_paths)):
        if i in visited:
            continue
        img1_path = image_paths[i]

        features1 = features[i]
        for j in range(i + 1, len(image_paths)):
            img2_path = image_paths[j]
            features2 = features[j]
            # So sánh đặc trưng của hai hình ảnh
            if np.array_equal(features1, features2):
                print(f"{img1_path} và {img2_path} giống nhau")
                visited.add(j)  # Đánh dấu hình ảnh thứ j đã được kiểm tra



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Các hình ảnh có đặc trưng giống nhau:
Cụm 2:
/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.916.png và /content/drive/MyDrive/Public/Honda/21522153.Honda.6.jpg giống nhau
/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.916.png và /content/drive/MyDrive/Public/Honda/22521293-22520105-22521624.Honda.450.jpg giống nhau
/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.1292.jpg và /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.724.jpg giống nhau
/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.153.jpg và /content/drive/MyDrive/Public/Honda/22521332.Honda.48.jpg giống nhau
/content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.153.jpg và /content/drive/MyDrive/Public/Honda/22520968-22520996-22520999-22520929-22521373.Honda.185.jpg giống nhau
/content/drive/MyDrive/Publ

#Hiển thị kết quả phân cụm (clustering)

In [22]:
from PIL import Image, ImageDraw
from PIL import Image
import csv
category_names = {
    0: 'Others',
    1: 'Honda',
    2: 'Suzuki',
    3: 'Yamaha',
    4: 'VinFast'
}
def extract_filename(file_path):
  # Split the file path into directories and filename using os.path.split()
  filename = file_path.split('/')
  return filename[5]+'/'+filename[6]

image_info= []
# Tạo figure cho biểu đồ
for cluster_id, image_paths in cluster_dict.items():
    for i in range(len(image_paths)):
      image_info.append([extract_filename(image_paths[i]), cluster_id])

#File Cluster
csv_file_path = '/content/drive/MyDrive/MotocycleDataset.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Viết tiêu đề cho file CSV
    writer.writerow(['ImageFullPath', 'ClusterID'])
    # Viết thông tin của mỗi ảnh vào file CSV
    for info in image_info:
        writer.writerow(info)

In [32]:
import matplotlib.pyplot as plt
import numpy as np  # For NumPy arrays
from PIL import Image  # For PIL Image objects
import random
FontSize = 24 #size chữ
# dùng để hiển thị
NumImgsPerRow = 10
ImgHeight = ImgWidth = 150
dtf = pd.read_csv('/content/drive/MyDrive/MotocycleDataset.csv')
# Tạo figure cho biểu đồ
fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(20, 10))
clusterID = [i for i in range(num_clusters)]
# Xử lý và hiển thị ảnh cho mỗi cluster
for ax, cluster in zip(axes, (clusterID[i] for i in range(0,num_clusters))):
    # Lọc ra các ảnh thuộc cluster hiện tại
    category_df = dtf[dtf['ClusterID'] == cluster]

    # Chọn ngẫu nhiên 10 ảnh
    selected_images = category_df.sample(n=min(NumImgsPerRow, len(category_df)))

    # Tạo một canvas trống để vẽ ảnh
    canvas = Image.new('RGB', (ImgWidth * (NumImgsPerRow + 1), ImgHeight), 'white')

    # Tạo một đối tượng để vẽ tên category
    draw = ImageDraw.Draw(canvas)
    draw.text((40, 50), f'Cluster {cluster}', fill='black')

    # Đọc và xử lý từng ảnh
    for i, (_, row) in enumerate(selected_images.iterrows(), start=1):
        if 'ImageFullPath' in row.index:
            img_path = os.path.join('/content/drive/MyDrive/Public/', row['ImageFullPath'])
        else:
            print(f"Warning: 'ImageFullPath' column not found for cluster{cluster}. Skipping image {i}.")
            continue

        # Attempt to open and resize the image
        try:
            img = Image.open(img_path)
            img = img.resize((ImgWidth, ImgHeight))
        except (FileNotFoundError, IOError):
            print(f"Warning: Image file not found or cannot be opened at '{img_path}'. Skipping image {i}.")
            continue
        canvas.paste(img, (i * ImgWidth, 0))

    # Chuyển canvas thành array và hiển thị trên subplot tương ứng
    ax.imshow(canvas)
    ax.axis('off')  # Tắt trục tọa độ

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.